# Generate Random Models

In testing, we often need to generate random models. `schematics` offer this capability via a feature called [Model Mocking]



[Model Mocking]: https://schematics.readthedocs.io/en/latest/usage/models.html#model-mocking

In [1]:
import schematics

class EnvironmentMetadata(schematics.Model):
    name = schematics.types.StringType(required=True, max_length=30)
    display_name = schematics.types.StringType(serialized_name="displayName", max_length=255, serialize_when_none=False)
    description = schematics.types.StringType(max_length=255, serialize_when_none=False)

In [2]:
environment_metadata = EnvironmentMetadata.get_mock_object()

In [3]:
environment_metadata.name

'hGy3yr4jj9k'

In [4]:
environment_metadata.display_name

In [5]:
environment_metadata.description

In [6]:
environment_metadata.to_primitive()

{'name': 'hGy3yr4jj9k'}

## Generate Nested Models

In [7]:
import json
import random

import schematics

class UnixUser(schematics.Model):
    alias = schematics.types.StringType(required=True)
    is_admin = schematics.types.BooleanType(default=False)
    
class Phone(schematics.Model):
    number = schematics.types.StringType(regex=r"\d{3}-\d{3}-\d{4}", required=True)
    kind = schematics.types.StringType(required=True,choices=["mobile", "work", "home", "other"])
    
class Contact(schematics.Model):
    name = schematics.types.StringType(required=True)
    
    # Nested: A single object
    unix_user = schematics.types.ModelType(UnixUser, serialized_name="unixUser", required=True)
    
    # Nested: Many objects
    phones = schematics.types.ListType(schematics.types.ModelType(Phone), default=[])

In [8]:
for _ in range(3):
    contact = Contact.get_mock_object()
    print("-" * 72)
    print(json.dumps(contact.to_primitive(), indent=4))

------------------------------------------------------------------------
{
    "name": "JwAs7V1adFieSQ",
    "unixUser": {
        "alias": "WFlR2",
        "is_admin": false
    },
    "phones": [
        {
            "number": "j83NvEHljOxrzq",
            "kind": "mobile"
        },
        {
            "number": "8X7",
            "kind": "mobile"
        },
        {
            "number": "UrJ",
            "kind": "home"
        },
        {
            "number": "pH2WGgfZsMsS",
            "kind": "work"
        },
        {
            "number": "zjkir",
            "kind": "mobile"
        },
        {
            "number": "TLgJfrq03VjIUXB",
            "kind": "other"
        },
        {
            "number": "ymXzGCkpnt",
            "kind": "mobile"
        },
        {
            "number": "8QPrBdgshE",
            "kind": "home"
        },
        {
            "number": "qT4y9",
            "kind": "work"
        },
        {
            "number": "ROWGbN",
        

## Overriding the Random Generator

If we need to override some fields, we can create a dictionary and pass that into `get_mock_object()`:

In [9]:
contact = Contact.get_mock_object(overrides={"name": "Johan"})
print(json.dumps(contact.to_primitive(), indent=4))

{
    "name": "Johan",
    "unixUser": {
        "alias": "KWUQAK",
        "is_admin": false
    },
    "phones": []
}


## Caveats

Note that in the example above, the random models might not be valid. For example, the phone number field does not satisfy the regular expression:

In [10]:
phone = Phone.get_mock_object()
print(phone.to_primitive())
try:
    phone.validate()
except schematics.exceptions.DataError as error:
    print(f"Error: {error}")

{'number': 'QWh6q0f6dCbxRLc', 'kind': 'other'}
Error: {"number": ["String value did not match validation regex."]}


For this reason, if we want to generate our own valid models, we need to do it ourselves and do not rely on `get_mock_object`.

In [11]:
def generate_random_phone_number():
    number = "-".join(
        str("".join(random.sample("0123456789", n)))
        for n in [3, 3, 4]
    )
    return number

class Phone(schematics.Model):
    number = schematics.types.StringType(regex=r"\d{3}-\d{3}-\d{4}", required=True)
    kind = schematics.types.StringType(required=True,choices=["mobile", "work", "home", "other"])
    
    @classmethod
    def get_mock_object(cls, context=None, overrides=None):
        obj = super().get_mock_object(context=context, overrides=overrides or {})
        try:
            obj.validate()
        except schematics.exceptions.DataError:
            obj.number = generate_random_phone_number()
        return obj

In [12]:
for _ in range(5):
    phone = Phone.get_mock_object()
    print(phone.to_primitive())

{'number': '970-428-3185', 'kind': 'home'}
{'number': '153-486-5206', 'kind': 'work'}
{'number': '652-834-5432', 'kind': 'other'}
{'number': '816-301-6072', 'kind': 'home'}
{'number': '584-847-1749', 'kind': 'mobile'}
